In [11]:
from glob import glob
import json
import re
import wikitextparser as wtp
import flair
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
import pandas as pd
from collections import defaultdict
import pickle
sns.set()

In [9]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2021-11-24 10:49:03,511 loading file C:\Users\lucas\.flair\models\sentiment-en-mix-distillbert_4.pt


In [10]:
char_sentiments = defaultdict(list)

seen_ids = {}
for fn in tqdm(glob('data/char_comments/*.njson')):
    char_name = os.path.basename(fn)[:-6].replace('_', ' ')
    comments = [json.loads(l) for l in open(fn, 'r', encoding='utf-8')]

    for comment in comments:
        text = comment.get('body', '')
        text = re.sub(r"\[.+?\]", "", text)
        text = re.sub(r"[\t\n\r]+", " ", text)
        s = flair.data.Sentence(text)
        flair_sentiment.predict(s)
        if len(s.labels) < 1:
            continue
        label = s.labels[0]
        sign = 1 if label.value == 'POSITIVE' else -1
        char_sentiments[char_name].append((comment['id'], sign*label.score))

  0%|          | 0/241 [00:00<?, ?it/s]

2021-11-24 10:49:59,707 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-11-24 10:50:04,523 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-11-24 10:50:50,823 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-11-24 10:50:54,299 Warning: An empty Sentence was created! Are there empty strings in your dataset?


In [12]:
with open('char_sentiments.pkl') as f:
    pickle.dump(dict(char_sentiments), f)

FileNotFoundError: [Errno 2] No such file or directory: 'char_sentiments.pkl'

In [202]:
df = pd.DataFrame({"char": k, "score": v} for k,V in char_sentiments.items() for v in V)

In [203]:
plt.figure(figsize=(7,60))
sns.boxplot(y="char", x="score", data=df, orient="h")
plt.show()

TypeError: Horizontal orientation requires numeric `x` variable.

<Figure size 504x4320 with 0 Axes>